In [3]:
# importing libraries
import cv2
import numpy as np
import matplotlib as mp
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import pickle
from matplotlib.path import Path
import matplotlib.patches as patches

### Camera Calibration Data

In [10]:
# following cells will have methods required for this project
# load object and image points saved after camera calibration done using chessboard images
obj_points = []
img_points = []
with open('obj_img_points.pickle', 'rb') as handle:
    a = pickle.load(handle)
    obj_points = a['objpoints']
    img_points = a['imgpoints']

## Helper Methods

This section include methods for sobel and all other action required

In [4]:
# appliying direction gradient on the image in x-axis
def performSobelX(image, sobel_kernal=3):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernal)
    
# appliying direction gradient on the image in y-axis
def performSobelY(image, sobel_kernal=3):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernal)

In [6]:
# applying the threshold on directional gradient
def performSobelGradientThreshold(image, axis='x', sobel_kernal=3, min_thresh=10, max_thresh=100):
    
    if axis is 'x':
        abs_sobel = np.absolute(performSobelX(image, sobel_kernal))
    else:
        abs_sobel = np.absolute(performSobelY(image, sobel_kernal))
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    s_binary = np.zeros_like(scaled_sobel)
    s_binary[(scaled_sobel >= min_thresh) & (scaled_sobel < max_thresh)] = 1
    
    return s_binary

In [5]:
# applying the threshold on the magnitude of the gradient
def performGradientMagnitudeThreshold(image, axis='x', sobel_kernal=3, mag_thresh=(0,255)):

    gradmag = None
    if axis is 'x':
        sobelx = performSobelX(image, sobel_kernal)
        gradmag = np.sqrt(sobelx**2)
    elif axis is 'y':
        sobely = performSobelY(image, sobel_kernal)
        gradmag = np.sqrt(sobely**2)
    elif axis is 'xy':
        sobelx = performSobelX(image)
        sobely = performSobelY(image)
        gradmag = np.sqrt(sobelx**2 + sobely**2)
    else:
        return null
    
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output
    
# applying threshold on the direction of the gradient
def performGradientDirectionThreshold(image, sobel_kernal=3, dir_thresh=(0, np.pi/2)):
    
    sobelx = performSobelX(image, sobel_kernal)
    sobley = performSobelY(image, sobel_kernal)
    
    absgraddir = np.arctan(np.absolute(sobely), np.absolute(sobelx))
    binary_img = np.zeros_like(absgraddir)
    binary_img[(absgraddir >= dir_thresh[0]) & (absgraddir < dir_thresh[1])] = 1
    
    return binary_img

In [7]:
# ploting two image side by side
def plot_images(img1, img2, title1, title2):
    plt.clf()
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1, cmap='gray')
    ax1.set_title(title1, fontsize=50)
    ax2.imshow(img2, cmap='gray')
    ax2.set_title(title2, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

In [8]:
# method to return the s-channel binary for given images
def hls_threshold(image, thresh=(0,255)):
    hls_img = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s = hls_img[:,:,2]
    output = np.zeros_like(s)
    output[(s > thresh[0]) & (s <= thresh[1])] = 1
    return output

In [9]:
def cal_undistort(orig_img, objpoints, imgpoints):
    gray_img = cv2.cvtColor(orig_img, cv2.COLOR_RGB2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray_img.shape[::-1], None, None)
    undist = cv2.undistort(orig_img, mtx, dist, None, mtx)
    return undist

In [ ]:
# this method will take the lane image and return the image with lane area, curvature and offset from center marked
def findAndPlotLaneLines(image):
    # 1. undistort the images
    # 2. color thresholding with s-channel
    # 3. perform various thresholding on the image
    # 3.a. sobelx thresholding
    # 3.b. gradient magnitude thresholding
    # 3.c. gradient direction thresholding
    # 4. create a combined binary of all above thresholding
    # 5. perform perspective transform
    # 6. increase the hot points in the image
    # 7. perform sliding windows search algorithm for finding lane lines and fitting a polygon
    # 8. mark the area of lane lines
    # 9. undistort the image back to starting perspective and return
    return null